#### Function Calling의 사용
- 이 코드에서 function calling은 모델이 적절한 시점에 외부 함수(get_current_weather)를 호출하도록 유도합니다. 모델이 What is the current weather in {user_location}?와 같은 질문에 응답할 때, 해당 함수가 정의되어 있으면 모델은 그 함수를 호출하여 날씨 정보를 가져오려 합니다.

- function_call="auto" 설정을 통해 GPT 모델은 적절한 함수가 있을 때 자동으로 그 함수를 호출할 수 있습니다. 여기서는 get_current_weather 함수 명세를 통해 사용자가 입력한 위치에 맞는 날씨 정보를 가져올 수 있도록 유도됩니다.

- 만약 모델이 함수 호출을 결정하면, 응답에 function_call이 포함되며, 이때 함수 이름과 그 인자가 함께 반환됩니다. 그런 후에 함수가 호출되고, 결과가 다시 모델에 제공됩니다.

이 과정에서 모델은 단순히 인자를 생성하고, 함수를 호출하는 것은 개발자의 책임입니다. OpenAI의 API는 실제 함수 실행을 수행하지 않으므로, 함수 호출 후 결과를 처리하고 다시 모델에 넘기는 과정을 수동으로 처리하게 됩니다.

In [ ]:
!pip install openai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.4/362.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00


https://home.openweathermap.org/

In [ ]:
import openai
import json
import requests

from openai import OpenAI
client = OpenAI(api_key='')

# 날씨 API 키 설정
weather_api_key = '2e1e37fd42dd84270c25d13cbba0c19f'

# 날씨 정보를 가져오는 함수 정의
def get_current_weather(location):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={weather_api_key}&units=metric"
    response = requests.get(url)
    return response.json()

# 사용자 요청 메시지
messages = [
    {'role': 'system', 'content': 'You are a helpful assistant.'},
    {'role': 'user', 'content': 'What is the current weather in New York?'}
]

# GPT-4 모델 호출
response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages = messages,
    functions = [{
        'name': 'get_current_weather',
        'description': 'Get the current weather for a specific location',
        'parameters': {
            'type': 'object',
            'properties': {
                'location': {
                    'type': 'string',
                    'description': 'The name of the city to get the weather for',
                },
            },
            'required': ['location'],
        },
    }],
    function_call = 'auto'
)

# 모델의 응답 메시지를 messages 리스트에 추가하고 출력
response_message = response.choices[0].message
messages.append(response_message)

print(response_message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{"location":"New York"}', name='get_current_weather'), tool_calls=None)


In [ ]:
# 도구 호출 여부 확인

function_call = response_message.function_call
if function_call:
    tool_function_name = function_call.name
    tool_arguments = json.loads(function_call.arguments)

    # 함수 호출 및 결과 처리
    if tool_function_name == 'get_current_weather':
        location = tool_arguments['location']
        weather_results = get_current_weather(location)

        # 함수 호출 결과 메시지 추가

        messages.append({
            'role': 'function',
            'name': tool_function_name,
            'content': json.dumps(weather_results) # json형식으로 반환
        })
        # print(messages)
        # 모델 재호출
        model_response_with_function_call = client.chat.completions.create(
            model = 'gpt-4o-mini',
            messages = messages
        )
        print(model_response_with_function_call.choices[0].message.content)
    else:
        print(f'Error: function {tool_function_name} does not exist')
else:
    # 도구 호출이 없는 경우 결과 반환
    print(response_message.content)

The current weather in New York is as follows:

- **Temperature**: 21.7°C (feels like 22.3°C)
- **Condition**: Mist
- **Humidity**: 92%
- **Visibility**: Approximately 4.8 km
- **Wind**: From the northeast at 4.6 m/s (gusts up to 7.7 m/s)
- **Cloud Cover**: 100%

Overall, it seems to be a misty day in New York.


In [ ]:
import openai
import json
import requests

from openai import OpenAI
client = OpenAI(api_key='')

# 날씨 API 키 설정
weather_api_key = '2e1e37fd42dd84270c25d13cbba0c19f'

# 날씨 정보를 가져오는 함수 정의
def get_current_weather(location):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={weather_api_key}&units=metric"
    response = requests.get(url)
    return response.json()

# 사용자 요청 메시지 초기화
messages = [
    {'role': 'system', 'content': 'You are a helpful assistant.'},
    {'role': 'user', 'content': 'plese provide the location to get the current weather infomation'}
]

# 사용자 입력 받기
user_location = input('Enter the location: ')
messages.append({'role': 'user', 'content': f'What is the current weather in {user_location}?'})

# GPT-4 모델 호출
response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages = messages,
    functions = [{
        'name': 'get_current_weather',
        'description': 'Get the current weather for a specific location',
        'parameters': {
            'type': 'object',
            'properties': {
                'location': {
                    'type': 'string',
                    'description': 'The name of the city to get the weather for',
                },
            },
            'required': ['location'],
        },
    }],
    function_call = 'auto'
)

# 모델의 응답 메시지를 messages 리스트에 추가하고 출력
response_message = response.choices[0].message
messages.append(response_message)

print(response_message)

Enter the location: seoul
ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Seoul"}', name='get_current_weather'), tool_calls=None)


In [ ]:
# 도구 호출 여부 확인

function_call = response_message.function_call
if function_call:
    tool_function_name = function_call.name
    tool_arguments = json.loads(function_call.arguments)

    # 함수 호출 및 결과 처리
    if tool_function_name == 'get_current_weather':
        location = tool_arguments['location']
        weather_results = get_current_weather(location)

        # 함수 호출 결과 메시지 추가

        messages.append({
            'role': 'function',
            'name': tool_function_name,
            'content': json.dumps(weather_results) # json형식으로 반환
        })
        # print(messages)
        # 모델 재호출
        model_response_with_function_call = client.chat.completions.create(
            model = 'gpt-4o-mini',
            messages = messages
        )
        print(model_response_with_function_call.choices[0].message.content)
    else:
        print(f'Error: function {tool_function_name} does not exist')
else:
    # 도구 호출이 없는 경우 결과 반환
    print(response_message.content)

The current weather in London is as follows:

- **Condition**: Scattered clouds
- **Temperature**: 14.25°C
- **Feels Like**: 13.77°C
- **Humidity**: 78%
- **Wind Speed**: 0.69 m/s from the west (286°)
- **Cloud Cover**: 40%

The temperature can vary, with a minimum of 11.62°C and a maximum of 16.12°C expected.

If you need more detailed weather information or forecasts, websites like [Weather.com](https://www.weather.com/) or [BBC Weather](https://www.bbc.co.uk/weather) can provide extensive details.


## How to call functions with model generated arguments

다음 예제에서는 모델에서 생성된 입력을 갖는 함수를 실행하는 방법을 보여주고 이를 사용하여 데이터베이스에 대한 질문에 답할 수 있는 에이전트를 구현합니다. 단순화를 위해 Chinook 샘플 데이터베이스를 사용하겠습니다 .

참고: 모델이 올바른 SQL을 생성하는 데 완벽하게 신뢰할 수 없기 때문에 프로덕션 환경에서 SQL 생성은 위험할 수 있습니다.

이 코드는 OpenAI의 GPT 모델을 사용하여 SQLite 데이터베이스에서 음악 관련 질문에 대한 답변을 SQL 쿼리로 변환한 후, 해당 쿼리를 실행하여 결과를 반환하는 방식으로 function calling을 구현한 예시입니다. 여기서는 사용자가 앨범 관련 질문을 하면, GPT-4가 질문을 SQL 쿼리로 변환하고, SQLite 데이터베이스를 조회하여 결과를 반환합니다.

specifying a function to execute sql queries

먼저 sqlite 데이터베이스에서 데이터를 추출하는데 유용한 유틸리티 함수를 정의
chinook.db 데이터베이스에 연결하여 sqlite를 통해 sql쿼리를 실행

In [ ]:
import sqlite3
from google.colab import drive

drive.mount('/content/drive')

# sqlite3.connect()를 통해 google drive에 있는 chinook.db sqlite 데이터베이스에 연결
conn = sqlite3.connect('/content/drive/MyDrive/kdt_240424/M9_LLM/dataset/Chinook.db')
print("Opened database successfully")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Opened database successfully


데이터베이스 테이블 및 열 정보 조회

- get_table_names, get_column_names 함수를 통해 데이터베이스의 테이블 및 열 이름을 가져옵니다.
- 이 정보는 나중에 GPT 모델이 SQL 쿼리를 생성할 때 사용할 스키마 정보를 제공하는 데 활용됩니다.

In [ ]:
# 데이터 베이스에서 테이블 목록을 추출하는 함수 sqlite_master 테이블에서 type이 table인 항목들의 이름을 가져와 리스트로 반환

def get_table_names(conn):
    """Return a list of table names."""
    table_names = []
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names

# PRAGMA table_info(table_name) 명령을 사용하여 테이블의 스키마 정보를 가져오고 컬럼 이름을 리스트로 반환
def get_column_names(conn, table_name):
    """Return a list of column names."""
    column_names = []
    columns = conn.execute(f"PRAGMA table_info({table_name});").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names

def get_database_info(conn):
    """Return a list of dicts containing the table name and columns for each table in the database."""
    table_dicts = []
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts


데이터베이스의 테이블과 열 정보를 문자열 형태로 저장하여, 나중에 SQL 쿼리 작성시 참수 할 수 있도록 합니다

In [ ]:
database_schema_dict = get_database_info(conn)
database_schema_string = '\n'.join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            'name': 'ask_database',
            'description': 'Use this function to answer user questions about music. Output should be a fully formed SQL query.',
            'parameters': {
                'type': 'object',
                'properties': {
                    'query': {
                        'type': 'string',
                        'description': f"""
                                SQL query extracting info to answer the user's question.
                                SQL should be written using this database schema:
                                {database_schema_string}
                                The query should be returned in plain text, not in JSON.
                                """,
                    }
                },
                'required': ['query']
            },
        }
    }
]

Executing SQL queries

이는 실제로 데이터베이스에 대한 쿼리를 실행하는 함수를 구현

In [ ]:
def ask_database(conn, query):
    """Function to query SQLite database with a provided SQL query."""
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

In [ ]:
# 사용자의 요청 메시지를 정의

messages = [{
    'role': 'user',
    'content': 'what is the name of the album with the most tracks'
}]

# 사용자의 질문에 대한 응답을 생성. tools 와 tool_choice 파라미터는 모델이 데이터베이스 쿼리를 인식하고 자동으로 도구를 선택할 수 있도록 도움
response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages = messages,
    tools = tools,
    tool_choice = 'auto' # 모델이 자동으로 함수 호출을 결정, SQL 쿼리로 변화할 필요가 있다고 판단하면 자동으로 ask_database 함수를 호출
)

# 모델의 응답 메시지를 messages 리스트에 추가하고 출력
response_message = response.choices[0].message
messages.append(response_message)
print(response_message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_oDeLHJW4oT4nJ9zf5wtkey2c', function=Function(arguments='{"query":"SELECT Album.Title FROM Album JOIN (SELECT AlbumId, COUNT(*) AS TrackCount FROM Track GROUP BY AlbumId ORDER BY TrackCount DESC LIMIT 1) AS MostTracks ON Album.AlbumId = MostTracks.AlbumId;"}', name='ask_database'), type='function')])


In [ ]:
# 모델 응답에서 도구 호출이 포함되어 있는지 확인하고 도구 호출이 있다면 도구 호출 ID 함수 이름 및 쿼리 문자열을 추출

tool_calls = response_message.tool_calls
if tool_calls:
    # if true the model will return the name of the tool / function to call the argument(s)
    tool_call_id = tool_calls[0].id
    tool_function_name = tool_calls[0].function.name
    tool_function_string = json.loads(tool_calls[0].function.arguments)['query']

    # 도구 호출 함수 이름이 'ask_database'인 경우 , ask_database 함수를 호출하여 데이터베이스 쿼리를 실행하고 결과를 messages 리스트에 추가
    if tool_function_name == 'ask_database':
        results = ask_database(conn, tool_function_string)

        messages.append({
            'role': 'tool',
            'tool_call_id': tool_call_id,
            'name': tool_function_name,
            'content': results
        })

        # 도구 호출 결과가 포함된 messages 리스트를 사용하여 모델을 다시 호출하고 최종 응답을 출력
        # Note that messages with role 'tool' must be a response to a preceding message with 'tool_call'
        model_response_with_function_call = client.chat.completions.create(
            model = 'gpt-4o-mini',
            messages = messages
        )  # get a new response from the model where it can see the function response
        print(model_response_with_function_call.choices[0].message.content)
        # 도구 호출 함수 이름이 ask_database 가 아닌 경우 오류 메시지를 출력하거나 도구 호출이 없으면 모델의 응답 내용을 바로 출력
    else:
        print(f'Error: function {tool_function_name} does not exist')
else:
    # Model did not identify a function to call, result can be returned to the user
    print(response_message.content)


The album with the most tracks is titled "Greatest Hits."
